<center><H2>МОСКОВСКИЙ ГОСУДАРСТВЕННЫЙ УНИВЕРСИТЕТ ИМЕНИ М.В. ЛОМОНОСОВА</H2>
<center><H2>ФАКУЛЬТЕТ ГЕОЛОГИИ</H2>
<center><H2>КАФЕДРА КРИСТАЛЛОГРАФИИ И КРИСТАЛЛОХИМИИ </H2>
<H2> </H2>
<H2> </H2>
<H2> </H2>
<H2> </H2>
<center><H3>Курсовая работа</H3>
<center><H2>Анализ релаксации атомов в твердых растворах с помощью языка программирования Python3</H2>
<center><H3> Выполнил:</H3>
<center><H2> Студент 2 курса </H2>
<center><H2> Трегубов Родион Дмитриевич </H2>
    <center><H3> Научный руководитель:</H3>
        <center><H2> доктор химических наук</H2>
            <center><H2>  профессор</H2>
<center><H2>Еремин Николай Николаевич</H2>


<center><H3>Введение</H3></center>
Оценка подвижности конкретного атома внутри сверхячейки является достаточно сложной и неопределенной задачей, так как при расчете меняются не только атомные координаты, но и сами параметры и формально выбранная точка начала координат не дает никакой информации и величине смещений атомов из их регулярных позиций. Очевидно, что при искажениях ячейки в целом, точки с большими координатами в долях ячейки будут менять координаты сильнее, чем точки приближенные к началу координат
В связи с этим в настоящей работе вводится величина «подвижность атома» ω.
В качестве исходных данных для расчета выступают исходный и конечный атомные массивы (координаты атомов и параметры ячейки в пространственной группе P1). 
<img src='relax.png'>
Рис 1 а) – выбор начала координат для каждого атома; б) нахождение наиболее подвижных атомов (зеленые атомы – конечная конфигурация).

Как видно из рисунка 1б наибольшими значениями ω будут характеризоваться атомы с максимальным собственным отклонением относительно всех остальных (атом No 18 на
рисунке 1б).

In [88]:
import warnings
warnings.simplefilter('ignore')
import numpy as np
from ase.io import read
from scipy.spatial import distance
import pandas as pd
from ase.build import stack
from ase.visualize import view
from ase import Atoms
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go

init_notebook_mode(connected=True)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
%run try.py

Мы используем функцию read из библиотеки ase для того чтобы напрямую считывать зи *.cif файлов, так же это приводит координаты к декартовым, что впоследствии сильно облегчает работу с ними.

In [89]:
atoms1 = read('MgSiO3_222.cif')
atoms2 = read('MgSiO3_222_opti.cif')

Вот так выглядит неопимизированная ячейка:

In [115]:
view(atoms1, viewer='ngl')

Оптимизированная:

In [91]:
view(atoms2, viewer='ngl')

Далее мы размножим наши ячейки

In [92]:
for i in range(len(atoms1)):
    for j in range(3):
        if np.abs(atoms1.get_scaled_positions()[i][j]-atoms2.get_scaled_positions()[i][j])>=0.75:
            if atoms1.get_scaled_positions()[i][j] > atoms2.get_scaled_positions()[i][j]:
                atoms1.positions[i]=atoms1.positions[i] - atoms1.cell[j]
            else: 
                atoms2.positions[i]=atoms2.positions[i] - atoms1.cell[j]
atoms1_x27 = razmnojenie(atoms1)
atoms2_x27 = razmnojenie(atoms2)

И вот что мы получим

In [110]:
view(atoms1_x27, viewer='ngl')

In [111]:
view(atoms2_x27,viewer='ngl')

In [95]:
def relax(r):
    w, neighbours1, neighbours2 = mobility(atoms1, atoms2,atoms1_x27,atoms2_x27,r)
    atoms1.tag, atoms = number_of_atoms(atoms1)
    df = pd.DataFrame({
        'Element' : list(atoms1.get_chemical_symbols()),
        '№' : list(atoms1.tag),
        'w' : list(w)
    })
    df=df.sort_values(by='w',ascending=False)
    df.to_csv()
    x = []
    for i in atoms:
        x.append([i,df[df['Element']==i].mean()[0]])
    return x

In [103]:
values = np.linspace(0.7,4.5,50)
x_y = []
for i in values:
    x_y.append(relax(i))

In [104]:
x_y=np.array(x_y)
trace0 = go.Scatter(
    x=values,
    y=x_y.T[1][0],
    name='Mg'
)
trace1 = go.Scatter(
    x=values,
    y=x_y.T[1][1],
    name='Si'
)
trace2 = go.Scatter(
    x=values,
    y=x_y.T[1][2],
    name='O'
)
data = [trace0, trace1, trace2]
layout = {'title': 'w от r'}
fig = go.Figure(data=data, layout=layout)
iplot(fig, show_link=True)

In [113]:
w, neighbours1, neighbours2 = mobility(atoms1, atoms2,atoms1_x27,atoms2_x27,2.7)

In [109]:
import scipy.spatial as ss
volume_points=[]
hull = ss.ConvexHull(neighbours2[50].positions)
hull.volume

8.035689226241495

In [100]:
view(neighbours2[1], viewer = 'ngl')

In [101]:
view(neighbours2[50], viewer = 'ngl')

In [105]:
neighbours2[1]

Atoms(symbols='O4MgO4', pbc=False, cell=[10.0592, 14.1695, 9.7426])

<H3></H3>
<H3></H3>
<H3></H3>
<H3></H3>
<H3></H3>
<center><H1>Cпасибо за внимание</H1>
<H3></H3>
<H3></H3>
<H3></H3>
<H3></H3>
<H3></H3>